In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import pandas as pd
import ast

In [28]:
df1=pd.read_csv('Results/happy_80images_H0_new.csv', sep=',')
df2=pd.read_csv('Results/angry_80images_H0_new.csv', sep=',')

In [48]:
x = []

for i in range(len(df1)):
    a = df1["content"][i].replace('\n', '').replace('  ', ',').replace(' ', ',')
    temparray = ast.literal_eval(a)
    x.append(temparray)
for i in range(len(df2)):
    a = df2["content"][i].replace('\n', '').replace('  ', ',').replace(' ', ',')
    temparray = ast.literal_eval(a)
    x.append(temparray)
    
print(len(x))
print(x)

y1 = np.ones(80)
y2 = np.zeros(80)
y_train = np.concatenate((y1, y2))
y_train

160
[[[0.0, 2.0, 2.23606801, 2.82842708, 3.0, 3.1622777, 3.46410155, 3.60555124, 3.7416575, 4.0, 4.12310553, 4.2426405, 4.47213602, 4.5825758, 4.69041586, 4.89897966, 5.0, 5.09901953, 5.19615221, 5.38516474, 5.47722578, 5.65685415, 5.74456263, 5.83095169, 6.0, 6.16441393, 6.3245554, 6.40312433, 6.55743837, 6.63324976, 6.70820379, 6.92820311, 7.0, 7.07106781, 7.21110249, 7.28010988, 7.34846926, 7.48331499, 7.54983425, 7.6157732, 8.24621105, 8.30662346, 9.1651516, 9.69536018, 12.1655254], [2304.0, 1759.0, 1143.0, 774.0, 664.0, 614.0, 563.0, 418.0, 404.0, 393.0, 330.0, 316.0, 223.0, 201.0, 186.0, 156.0, 137.0, 136.0, 135.0, 92.0, 91.0, 88.0, 79.0, 75.0, 62.0, 57.0, 43.0, 38.0, 37.0, 34.0, 29.0, 28.0, 23.0, 21.0, 17.0, 15.0, 14.0, 10.0, 9.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]], [[0.0, 2.0, 2.23606801, 2.82842708, 3.0, 3.1622777, 3.46410155, 3.60555124, 3.7416575, 4.0, 4.12310553, 4.2426405, 4.35889912, 4.47213602, 4.5825758, 4.69041586, 4.89897966, 5.0, 5.09901953, 5.38516474, 5.47722578, 5.656

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [47]:
#create fake training data as indexes of stable ranks
x_train = np.arange(160)
x_train.shape

(160,)

In [78]:
def srank_kernel(X, Y):
    """
    We create a custom stable rank kernel
    ref: JENS AGERBERG, Statistical Learning and Analysis on Homology-Based Features
    X,Y: stable rank array input data
    """
    t_union = np.union1d(X[0], Y[0])
    tY_diff = np.setdiff1d(Y[0], X[0]) #t in Y but not in X
    tX_diff = np.setdiff1d(X[0], Y[0]) #t in X but not in Y
    M1 = np.zeros(len(t_union))
    M2 = np.zeros(len(t_union))
    x_offset = 0
    y_offset = 0
    for i in range(len(t_union)):
        if t_union[i] <= X[0][-1] and t_union[i] == X[0][i-x_offset]:
            M1[i] = X[1][i-x_offset]
        else:
            x_offset = x_offset + 1
            M1[i] = X[1][i-x_offset]
            
        if t_union[i] <= Y[0][-1] and t_union[i] == Y[0][i-y_offset]:
            M2[i] = Y[1][i-y_offset]
        else:
            y_offset = y_offset + 1
            M2[i] = Y[1][i-y_offset]
    t_diff = np.ediff1d(t_union)
    return np.dot(np.multiply(M1[0:len(M1)-1], M2[0:len(M2)-1]), t_diff)

#this requires return a Gram matrix
def proxy_kernel(X,Y):
    gram_matrix = np.zeros((len(X), len(Y)))
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            gram_matrix[i, j] = srank_kernel(x, y)
    return gram_matrix

# we create an instance of SVM and fit out data.
clf = svm.SVC(kernel=proxy_kernel)
clf.fit(x, y_train)

enter srank_kernel
45
45


SVC(kernel=<function proxy_kernel at 0x7f4c698605e0>)